In [1]:
import cv2
import numpy as np
import os
import re
from sklearn.cluster import KMeans
%matplotlib inline 

In [2]:
dataSet = []
person = []
horiSet = []
train_set_x = []
count = 0
folder = "/home/jeet/ML face images train"
for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
#         print(img)
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.resize(img, (200,200))
        sift = cv2.xfeatures2d.SIFT_create()
        kp, des = sift.detectAndCompute(img,None)
        kmeans = KMeans(n_clusters=1, random_state=0).fit(des)
#         print(kmeans.cluster_centers_.shape)
        train_set_x.append(kmeans.cluster_centers_)
#         person.append(numbers[0])
        img = np.ravel(img)
        if img is not None:
#             for desc in des: 
            numbers = re.findall('\d+',filename)
            numbers = list(map(int,numbers))
            person.append(numbers[0])
            horiSet.append(img.T)
#             dataSet.append(desc)
            count = count + 1

print(np.array(train_set_x).shape)      
dataSet_test = []
person_test = []
horiSet_test = []
test_set_x = []
folder = "/home/jeet/ML face images test"
for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.resize(img, (200, 200))
        sift = cv2.xfeatures2d.SIFT_create()
        kp, des = sift.detectAndCompute(img,None)
        #kp will be a list of keypoints and des is a numpy array of shape Number_of_Keypoints×128.
        kmeans = KMeans(n_clusters=1, random_state=0).fit(des)
        test_set_x.append(kmeans.cluster_centers_)
        img = np.ravel(img)
        if img is not None:
#             for desc in des: 
            numbers = re.findall('\d+',filename)
            numbers = list(map(int,numbers))
            person_test.append(numbers[0])
            horiSet_test.append(img.T)
#             dataSet_test.append(desc)
            count = count + 1
print(np.array(test_set_x).shape , np.array(person_test).shape)

(418, 1, 128)
(140, 1, 128) (140,)


In [3]:
train_set_x = np.reshape(train_set_x,(418,128))
test_set_x = np.reshape(test_set_x,(140,128))

In [4]:
import pandas as pd

#Getting data in desired format and mapping the lables
dataSet = np.array(train_set_x)
train = pd.DataFrame(dataSet)

person = np.array(person)
train['id'] = person.transpose()

unique_person = train['id']
unique_person = np.array(unique_person)
unique_person = np.unique(unique_person)
key = range(len(unique_person))

roll_dict = dict( zip(unique_person, key))

mapped_roll = np.vectorize(roll_dict.get)(person)
train['id'] = mapped_roll.transpose()

y = train['id'].values.astype('int64')
images = train.drop(['id'], axis=1, inplace=False)
x = (images.values).astype('uint8')

X_train = x
Y_train = y

dataSet_test = np.array(test_set_x)
test = pd.DataFrame(dataSet_test)

person_test = np.array(person_test)
test['id'] = person_test.transpose()

mapped_roll_test = np.vectorize(roll_dict.get)(person_test)
test['id'] = mapped_roll_test.transpose()
print(test['id'])

# temp = np_utils.to_categorical(test['id'])
# print(temp[2])
# print(temp.shape)


y_test = test['id'].values.astype('int64')
images = test.drop(['id'], axis=1, inplace=False)
x_test = (images.values).astype('uint8')

X_test = x_test
Y_test = y_test



0      29
1      32
2       4
3      14
4      34
5      35
6      25
7      29
8       6
9      18
10     25
11     19
12     21
13     24
14     28
15     33
16     10
17      2
18      7
19      9
20     23
21     31
22     15
23     12
24      6
25     21
26     20
27      8
28     26
29     34
       ..
110    35
111    31
112    15
113    23
114    17
115     9
116    33
117    19
118    26
119    33
120    23
121    18
122    27
123    14
124    10
125     0
126    20
127    31
128     6
129    17
130    34
131     8
132    35
133    26
134    18
135     6
136    30
137    14
138     9
139    13
Name: id, Length: 140, dtype: int64


In [6]:
folder = "/home/jeet/ML face images test"
min_arr = []
count=0
for filename in os.listdir(folder):
        distance=[]
        img = cv2.imread(os.path.join(folder,filename))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        img = cv2.resize(img, (200, 200))
        sift = cv2.xfeatures2d.SIFT_create()
        kp, des = sift.detectAndCompute(img,None)
        #kp will be a list of keypoints and des is a numpy array of shape Number_of_Keypoints×128.
        kmeans = KMeans(n_clusters=1, random_state=0).fit(des)
        data = kmeans.cluster_centers_
        img = np.ravel(img)
        print("file : ",filename)
        
        for train_data in X_train:
            dist = np.linalg.norm(data-train_data)
            distance.append(dist)
#         min_dist = distance.index(np.min(distance))
#         print(min_dist)
            min_arr.append(distance.index(np.min(distance)))
        dis = distance.index(np.min(distance))
        print(Y_train[dis])
        
        for roll in roll_dict:
            if str(roll) in filename:
                print(roll)
                print(roll_dict[roll])
                
                if roll_dict[roll] == Y_train[dis]:
                    count = count+1
                    
print(count)
    

file :  201501086O_disgust.jpg
4
201501086
29
file :  201501095O_fear.jpg
24
201501095
32
file :  1849004O_fear.JPG
26
1849004
4
file :  201501028O_disgust.JPG
16
201501028
14
file :  201501104O_fear.jpg
34
201501104
34
file :  201501109O_fear.jpg
35
201501109
35
file :  201501067_disgust.jpg
10
201501067
25
file :  201501086O_fear.jpg
16
201501086
29
file :  201501004_disgust.JPG
6
201501004
6
file :  201501038_fear.jpg
20
201501038
18
file :  201501067O_disgust.jpg
25
201501067
25
file :  201501039O_fear.jpg
19
201501039
19
file :  201501053_fear.jpg
21
201501053
21
file :  201501060_fear.jpg
8
201501060
24
file :  201501077_fear.jpg
27
201501077
28
file :  201501097O_fear.jpg
1
201501097
33
file :  201501011_disgust.jpg
10
201501011
10
file :  1744003O_disgust.jpg
2
1744003
2
file :  201501007O_disgust.jpg
7
201501007
7
file :  201501009_fear.jpg
9
201501009
9
file :  201501055_fear.jpg
7
201501055
23
file :  201501091O_disgust.jpg
26
201501091
31
file :  201501031_disgust.jpg
15
20